# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.119209e+02     1.772026e+00
 * time: 0.7002918720245361
     1     1.072308e+01     9.614443e-01
 * time: 3.1032118797302246
     2    -1.200084e+01     1.157315e+00
 * time: 3.840285062789917
     3    -3.414897e+01     8.932160e-01
 * time: 4.805388927459717
     4    -4.789867e+01     5.823769e-01
 * time: 5.774584054946899
     5    -5.695287e+01     1.973861e-01
 * time: 7.148752927780151
     6    -5.980205e+01     8.775398e-02
 * time: 7.791231870651245
     7    -6.084986e+01     5.857470e-02
 * time: 8.434614896774292
     8    -6.126330e+01     6.352012e-02
 * time: 9.077996969223022
     9    -6.161319e+01     3.422309e-02
 * time: 9.95672607421875
    10    -6.184996e+01     2.610041e-02
 * time: 10.59305191040039
    11    -6.200915e+01     2.253048e-02
 * time: 11.234220027923584
    12    -6.207534e+01     1.925230e-02
 * time: 11.912605047225952
    13    -6.213686e+01     1.431383e-02
 * time: 12.552502870559692
   

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671063
    AtomicLocal         -18.8557688
    AtomicNonlocal      14.8522660
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485372 
    Xc                  -19.3336820

    total               -62.261666460600
